<a href="https://colab.research.google.com/github/crlsyajie/Machine-Learning-Models/blob/main/BARAKOMODEL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python-headless --quiet

import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt


In [ ]:
base_folder = "/content/drive/MyDrive/kapeng_barako"
img_width, img_height = 128, 128
input_shape = (img_width, img_height, 3)
batch_size = 32
epochs = 20

COLOR_MODE = 'RGB'

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_gen = datagen.flow_from_directory(
    base_folder,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='input',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    base_folder,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='input',
    subset='validation'
)


Found 7209 images belonging to 4 classes.
Found 801 images belonging to 4 classes.


In [ ]:
def build_autoencoder():
    input_img = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

autoencoder = build_autoencoder()
autoencoder.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_4 (UpSampling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_5 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 128, 128, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,067 (203.39 KB)

 Trainable params: 52,067 (203.39 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
checkpoint_path = "/content/drive/MyDrive/coffee_autoencoder.h5"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(checkpoint_path, save_best_only=True)
]

history = autoencoder.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - loss: 0.0258

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


226/226 ━━━━━━━━━━━━━━━━━━━━ 1824s 8s/step - loss: 0.0257 - val_loss: 0.0025
Epoch 2/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.0022

226/226 ━━━━━━━━━━━━━━━━━━━━ 782s 3s/step - loss: 0.0022 - val_loss: 0.0020
Epoch 3/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.0017

...

226/226 ━━━━━━━━━━━━━━━━━━━━ 777s 3s/step - loss: 0.0010 - val_loss: 8.4875e-04
Epoch 12/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 9.5301e-04

226/226 ━━━━━━━━━━━━━━━━━━━━ 790s 3s/step - loss: 9.5298e-04 - val_loss: 8.0576e-04
Epoch 13/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 9.5688e-04

226/226 ━━━━━━━━━━━━━━━━━━━━ 791s 4s/step - loss: 9.5678e-04 - val_loss: 7.7362e-04
Epoch 14/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 752s 3s/step - loss: 9.8440e-04 - val_loss: 7.8319e-04
Epoch 15/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 755s 3s/step - loss: 9.3049e-04 - val_loss: 8.8435e-04
Epoch 16/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 8.8665e-04

226/226 ━━━━━━━━━━━━━━━━━━━━ 803s 3s/step - loss: 8.8665e-04 - val_loss: 7.6084e-04
Epoch 17/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 754s 3s/step - loss: 9.0300e-04 - val_loss: 7.8529e-04
Epoch 18/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 8.9975e-04

226/226 ━━━━━━━━━━━━━━━━━━━━ 755s 3s/step - loss: 8.9970e-04 - val_loss: 7.3806e-04
Epoch 19/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 757s 3s/step - loss: 8.7146e-04 - val_loss: 7.6935e-04
Epoch 20/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 775s 3s/step - loss: 8.4004e-04 - val_loss: 7.7738e-04


In [ ]:
def preprocess_image(img_path, target_size=(128, 128), show_plot=True):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError("Image not found or unreadable")

    original = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError("No object found in image")

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped = img[y:y+h, x:x+w]

    resized = cv2.resize(cropped, target_size)

    if COLOR_MODE == 'HSV':
        converted = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)
    elif COLOR_MODE == 'LAB':
        converted = cv2.cvtColor(resized, cv2.COLOR_BGR2LAB)
    else:
        converted = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)

    sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(converted, -1, sharpen_kernel)

    if show_plot:
        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(original)
        plt.title("Original")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(sharpened)
        plt.title(f"Processed ({COLOR_MODE})")
        plt.axis("off")
        plt.show()

    normalized = sharpened.astype("float32") / 255.0
    return normalized


In [ ]:
def is_coffee(img_path, threshold=0.039):
    try:
        img_array = preprocess_image(img_path)
        img_array = np.expand_dims(img_array, axis=0)

        reconstructed = autoencoder.predict(img_array)
        loss = np.mean((img_array - reconstructed) ** 2)

        print(f"🔍 Reconstruction Loss: {loss:.6f}")
        return loss < threshold
    except Exception as e:
        print(f"⚠️ Error: {e}")
        return False


In [ ]:
from google.colab import files
from IPython.display import display, Image as IPImage

autoencoder = load_model("/content/drive/MyDrive/coffee_autoencoder.h5", compile=False)

uploaded = files.upload()

for filename in uploaded.keys():
    display(IPImage(filename))
    if is_coffee(filename):
        print("✅ Detected: Coffee Bean")
    else:
        print("❌ Detected: Not Coffee")


NameError: name 'load_model' is not defined

In [ ]:
files.download('/content/drive/MyDrive/coffee_autoencoder.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>